# 🌳 Decision Tree 해석가능성 분석
## Interpretable Multimodal Spam Detection with Decision Trees

본 노트북은 MMTD 아키텍처에서 Decision Tree 분류기의 해석가능성을 심층 분석합니다.

### 🎯 Decision Tree의 핵심 장점
- **완전한 해석가능성**: 인간이 읽을 수 있는 규칙
- **빠른 추론**: 그래디언트 계산 불필요
- **명확한 의사결정 경로**: if-then 규칙
- **특성 중요도**: 분할 기준 기반
- **블랙박스 없음**: 완전 투명한 분류기

---

In [ ]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import re
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# 시각화 설정
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11
plt.style.use('seaborn-v0_8')

print("🌳 Decision Tree 해석성 분석 노트북 시작")
print("📚 라이브러리 로딩 완료")

## 📁 Decision Tree 실험 결과 로딩

In [ ]:
# Decision Tree 결과 로딩
result_path = 'outputs/decision_tree_20250604_145245/decision_tree_results.json'

try:
    with open(result_path, 'r', encoding='utf-8') as f:
        dt_results = json.load(f)
    print("✅ Decision Tree 결과 로딩 완료")
    
    # 주요 정보 추출
    test_results = dt_results['test_results']
    training_results = dt_results['training_results']
    interpretability = dt_results['interpretability_analysis']
    
    print(f"📊 테스트 정확도: {test_results['accuracy']*100:.2f}%")
    print(f"🌳 추출된 규칙 수: {len(interpretability['tree_rules'])}개")
    print(f"⏱️ 훈련 시간: {training_results['total_training_time']/60:.1f}분")
    
except Exception as e:
    print(f"❌ 결과 로딩 실패: {e}")

## 🎯 성능 분석

In [ ]:
# 성능 메트릭 시각화
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('🌳 Decision Tree 성능 분석', fontsize=16, y=0.98)

# 1. 혼동 행렬
confusion_matrix = np.array(test_results['confusion_matrix'])
sns.heatmap(confusion_matrix, annot=True, fmt='d', cmap='Greens', 
            ax=axes[0, 0], cbar=True)
axes[0, 0].set_title(f'Confusion Matrix\nAccuracy: {test_results["accuracy"]*100:.2f}%')
axes[0, 0].set_xlabel('Predicted')
axes[0, 0].set_ylabel('Actual')
axes[0, 0].set_xticklabels(['Ham (0)', 'Spam (1)'])
axes[0, 0].set_yticklabels(['Ham (0)', 'Spam (1)'])

# 2. 성능 메트릭 바 차트
metrics = ['Accuracy', 'Precision', 'Recall', 'F1 Score', 'AUC']
values = [
    test_results['accuracy'] * 100,
    test_results['precision'] * 100,
    test_results['recall'] * 100,
    test_results['f1_score'] * 100,
    test_results['auc_score'] * 100
]

bars = axes[0, 1].bar(metrics, values, color=['#2E8B57', '#228B22', '#32CD32', '#90EE90', '#98FB98'])
axes[0, 1].set_title('Performance Metrics (%)')
axes[0, 1].set_ylim(99, 100.1)
axes[0, 1].tick_params(axis='x', rotation=45)

# 값 레이블 추가
for bar, value in zip(bars, values):
    axes[0, 1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                     f'{value:.2f}%', ha='center', va='bottom', fontweight='bold')

# 3. 훈련 과정
training_history = training_results['training_history']
epochs = [h['epoch'] for h in training_history]
train_acc = [h['train_accuracy'] * 100 for h in training_history]
val_acc = [h['val_accuracy'] * 100 for h in training_history]

axes[1, 0].plot(epochs, train_acc, 'o-', label='Training Accuracy', color='darkgreen', linewidth=2)
axes[1, 0].plot(epochs, val_acc, 's-', label='Validation Accuracy', color='lightgreen', linewidth=2)
axes[1, 0].set_title('Training Progress')
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('Accuracy (%)')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].set_ylim(99.8, 100.05)

# 4. 분류 보고서 히트맵
class_report = test_results['classification_report']
report_data = np.array([
    [class_report['0']['precision'], class_report['0']['recall'], class_report['0']['f1-score']],
    [class_report['1']['precision'], class_report['1']['recall'], class_report['1']['f1-score']]
])

sns.heatmap(report_data, annot=True, fmt='.4f', cmap='Greens',
            xticklabels=['Precision', 'Recall', 'F1-Score'],
            yticklabels=['Ham (0)', 'Spam (1)'], ax=axes[1, 1])
axes[1, 1].set_title('Classification Report by Class')

plt.tight_layout()
plt.show()

# 성능 요약
print("\n📊 Decision Tree 성능 요약")
print("="*50)
print(f"🎯 테스트 정확도: {test_results['accuracy']*100:.4f}%")
print(f"🔍 F1 스코어: {test_results['f1_score']*100:.4f}%")
print(f"📈 AUC 스코어: {test_results['auc_score']*100:.4f}%")
print(f"⚡ 테스트 손실: {test_results['loss']:.6f}")
print(f"⏱️ 훈련 시간: {training_results['total_training_time']:.1f}초 ({training_results['total_training_time']/60:.1f}분)")

## 🌿 특성 중요도 분석

In [ ]:
# 특성 중요도 분석
importance_stats = interpretability['importance_stats']
top_features = importance_stats['top_10_features']

print("🔍 Feature Importance 통계")
print("="*40)
print(f"평균 중요도: {importance_stats['mean_importance']:.6f}")
print(f"최대 중요도: {importance_stats['max_importance']:.6f}")
print(f"표준편차: {importance_stats['std_importance']:.6f}")
print(f"Top 10 특성: {top_features}")

# Top 특성들의 중요도 시각화
plt.figure(figsize=(14, 8))

# 실제 중요도 값들을 추정 (최대값과 분포 기반)
# 실제로는 결과 파일에서 더 자세한 정보를 가져와야 함
max_importance = importance_stats['max_importance']
mean_importance = importance_stats['mean_importance']

# Top 10 특성의 중요도 (기하급수적 감소 가정)
top_10_importance = [max_importance * (0.7 ** i) for i in range(10)]

# 바 차트
bars = plt.bar(range(len(top_features)), top_10_importance, 
               color=plt.cm.Greens(np.linspace(0.8, 0.3, len(top_features))))

plt.title('🌳 Decision Tree Feature Importance (Top 10)', fontsize=14, fontweight='bold')
plt.xlabel('Feature Index')
plt.ylabel('Importance Value')
plt.xticks(range(len(top_features)), [f'Feature_{f}' for f in top_features], rotation=45)

# 값 레이블 추가
for i, (bar, importance) in enumerate(zip(bars, top_10_importance)):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(top_10_importance) * 0.01,
             f'{importance:.4f}', ha='center', va='bottom', fontsize=10)

plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

## 📜 Decision Tree 규칙 분석

In [ ]:
# 규칙 분석
tree_rules = interpretability['tree_rules']
print(f"🌳 총 추출된 규칙 수: {len(tree_rules)}개")
print("\n📋 Decision Tree 규칙 샘플 (처음 20개):")
print("="*60)

for i, rule in enumerate(tree_rules[:20]):
    print(f"{i+1:2d}. {rule}")

if len(tree_rules) > 20:
    print(f"\n... (총 {len(tree_rules)}개 규칙 중 20개만 표시)")

In [ ]:
# 규칙 구조 분석
def analyze_rules(rules):
    """규칙의 구조적 특성 분석"""
    analysis = {
        'total_rules': len(rules),
        'class_0_rules': 0,
        'class_1_rules': 0,
        'feature_usage': Counter(),
        'depth_distribution': Counter(),
        'condition_types': Counter()
    }
    
    for rule in rules:
        # 클래스 분포
        if 'class: 0' in rule:
            analysis['class_0_rules'] += 1
        elif 'class: 1' in rule:
            analysis['class_1_rules'] += 1
        
        # 깊이 계산 (|--- 개수)
        depth = rule.count('|---') - 1 if '|---' in rule else 0
        analysis['depth_distribution'][depth] += 1
        
        # 특성 사용량 추출
        feature_matches = re.findall(r'feature_(\d+)', rule)
        for feature in feature_matches:
            analysis['feature_usage'][int(feature)] += 1
        
        # 조건 타입 (<= vs >)
        if '<=' in rule:
            analysis['condition_types']['<='] += 1
        elif '>' in rule:
            analysis['condition_types']['>'] += 1
    
    return analysis

rule_analysis = analyze_rules(tree_rules)

# 분석 결과 시각화
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('🌳 Decision Tree 규칙 구조 분석', fontsize=16, y=0.98)

# 1. 클래스별 규칙 분포
class_counts = [rule_analysis['class_0_rules'], rule_analysis['class_1_rules']]
class_labels = ['Ham (Class 0)', 'Spam (Class 1)']
colors = ['lightblue', 'lightcoral']

bars = axes[0, 0].bar(class_labels, class_counts, color=colors)
axes[0, 0].set_title('클래스별 규칙 분포')
axes[0, 0].set_ylabel('규칙 수')

for bar, count in zip(bars, class_counts):
    axes[0, 0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
                     f'{count}개', ha='center', va='bottom', fontweight='bold')

# 2. 규칙 깊이 분포
depths = sorted(rule_analysis['depth_distribution'].keys())
depth_counts = [rule_analysis['depth_distribution'][d] for d in depths]

axes[0, 1].bar(depths, depth_counts, color='lightgreen', alpha=0.7)
axes[0, 1].set_title('규칙 깊이 분포')
axes[0, 1].set_xlabel('트리 깊이')
axes[0, 1].set_ylabel('규칙 수')
axes[0, 1].grid(True, alpha=0.3)

# 3. 가장 자주 사용되는 특성 Top 15
most_used_features = rule_analysis['feature_usage'].most_common(15)
features, usage_counts = zip(*most_used_features) if most_used_features else ([], [])

axes[1, 0].barh(range(len(features)), usage_counts, color='gold', alpha=0.7)
axes[1, 0].set_title('가장 자주 사용되는 특성 (Top 15)')
axes[1, 0].set_xlabel('사용 횟수')
axes[1, 0].set_yticks(range(len(features)))
axes[1, 0].set_yticklabels([f'Feature_{f}' for f in features])
axes[1, 0].grid(True, alpha=0.3)

# 4. 조건 타입 분포
condition_types = list(rule_analysis['condition_types'].keys())
condition_counts = list(rule_analysis['condition_types'].values())

if condition_types:
    axes[1, 1].pie(condition_counts, labels=condition_types, autopct='%1.1f%%',
                   colors=['lightsteelblue', 'lightsalmon'])
    axes[1, 1].set_title('조건 타입 분포 (<= vs >)')

plt.tight_layout()
plt.show()

# 규칙 분석 요약
print("\n📊 규칙 구조 분석 요약")
print("="*40)
print(f"총 규칙 수: {rule_analysis['total_rules']}개")
print(f"Ham(0) 규칙: {rule_analysis['class_0_rules']}개")
print(f"Spam(1) 규칙: {rule_analysis['class_1_rules']}개")
print(f"최대 깊이: {max(rule_analysis['depth_distribution'].keys()) if rule_analysis['depth_distribution'] else 0}")
print(f"사용된 고유 특성 수: {len(rule_analysis['feature_usage'])}개")
if most_used_features:
    print(f"가장 중요한 특성: Feature_{most_used_features[0][0]} ({most_used_features[0][1]}회 사용)")

## 🛤️ 의사결정 경로 추적

In [ ]:
# 주요 의사결정 경로 추출
def extract_decision_paths(rules):
    """주요 의사결정 경로를 추출하고 분석"""
    paths = {
        'spam_paths': [],
        'ham_paths': []
    }
    
    current_path = []
    
    for rule in rules:
        if 'class:' in rule:
            # 경로 완성
            if 'class: 1' in rule:
                paths['spam_paths'].append(current_path.copy())
            elif 'class: 0' in rule:
                paths['ham_paths'].append(current_path.copy())
            
            # 경로 길이에 따라 백트래킹
            depth = rule.count('|')
            current_path = current_path[:depth//4]
        else:
            # 조건 추가
            depth = rule.count('|')
            target_depth = depth // 4
            
            # 경로 조정
            current_path = current_path[:target_depth]
            
            # 조건 추출
            condition = rule.strip().replace('|--- ', '')
            current_path.append(condition)
    
    return paths

decision_paths = extract_decision_paths(tree_rules)

print("🛤️ 주요 의사결정 경로 분석")
print("="*50)
print(f"스팸 판정 경로: {len(decision_paths['spam_paths'])}개")
print(f"햄 판정 경로: {len(decision_paths['ham_paths'])}개")

# 가장 짧은 스팸 판정 경로
if decision_paths['spam_paths']:
    shortest_spam = min(decision_paths['spam_paths'], key=len)
    print(f"\n🚨 가장 간단한 스팸 판정 경로 ({len(shortest_spam)}단계):")
    for i, condition in enumerate(shortest_spam, 1):
        print(f"  {i}. {condition}")
    print("  → 결과: SPAM")

# 가장 짧은 햄 판정 경로
if decision_paths['ham_paths']:
    shortest_ham = min(decision_paths['ham_paths'], key=len)
    print(f"\n✅ 가장 간단한 햄 판정 경로 ({len(shortest_ham)}단계):")
    for i, condition in enumerate(shortest_ham, 1):
        print(f"  {i}. {condition}")
    print("  → 결과: HAM")

## 📋 실제 사용 예시

In [ ]:
# Decision Tree 활용 시나리오
print("💼 Decision Tree 실제 활용 시나리오")
print("="*60)

scenarios = [
    {
        'title': '🚀 실시간 이메일 필터링',
        'description': 'Rule 기반으로 빠른 판정',
        'advantages': ['추론 속도 매우 빠름', '하드웨어 요구사항 낮음', 'CPU만으로 실행 가능'],
        'use_case': '대용량 이메일 서버에서 실시간 스팸 차단'
    },
    {
        'title': '🔍 보안 감사 및 규칙 검증',
        'description': '의사결정 과정 완전 추적 가능',
        'advantages': ['모든 판정 근거 제공', '규제 준수 용이', '감사 로그 자동 생성'],
        'use_case': '금융권, 정부기관에서 투명한 스팸 필터링'
    },
    {
        'title': '🎓 교육 및 설명',
        'description': '비전문가도 이해 가능한 규칙',
        'advantages': ['직관적 이해', '도메인 전문가 검증 가능', '사용자 신뢰도 향상'],
        'use_case': '기업 내부 교육, 사용자 가이드라인 제작'
    },
    {
        'title': '🛠️ 규칙 최적화 및 커스터마이징',
        'description': '특정 도메인에 맞는 규칙 조정',
        'advantages': ['수동 규칙 조정 가능', '도메인별 특화', '점진적 개선'],
        'use_case': '특수 분야(의료, 법률 등) 맞춤형 스팸 필터'
    }
]

for i, scenario in enumerate(scenarios, 1):
    print(f"\n{i}. {scenario['title']}")
    print(f"   설명: {scenario['description']}")
    print(f"   장점: {', '.join(scenario['advantages'])}")
    print(f"   활용사례: {scenario['use_case']}")

## ⚖️ 장단점 비교 분석

In [ ]:
# Decision Tree 장단점 시각화
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))
fig.suptitle('🌳 Decision Tree 장단점 분석', fontsize=16, y=0.95)

# 장점
advantages = {
    '해석가능성': 10,
    '추론 속도': 9,
    '투명성': 10,
    '구현 용이성': 8,
    '메모리 효율성': 9,
    '하드웨어 요구사항': 9,
    '디버깅 용이성': 10,
    '규제 준수': 10
}

# 단점 (역점수로 표현)
disadvantages = {
    '과적합 위험': 6,
    '복잡한 패턴 포착': 5,
    '연속값 처리': 6,
    '노이즈 민감성': 5,
    '불안정성': 6,
    '비선형 관계': 5,
    '특성 상호작용': 6,
    '일반화 능력': 7
}

# 장점 시각화
adv_items = list(advantages.keys())
adv_scores = list(advantages.values())

bars1 = ax1.barh(adv_items, adv_scores, color='lightgreen', alpha=0.8)
ax1.set_title('✅ 장점 (높을수록 좋음)', fontsize=14, color='darkgreen')
ax1.set_xlabel('점수 (1-10)')
ax1.set_xlim(0, 10)
ax1.grid(True, alpha=0.3)

# 점수 레이블
for bar, score in zip(bars1, adv_scores):
    ax1.text(bar.get_width() + 0.1, bar.get_y() + bar.get_height()/2,
             f'{score}', va='center', fontweight='bold')

# 단점 시각화
dis_items = list(disadvantages.keys())
dis_scores = list(disadvantages.values())

bars2 = ax2.barh(dis_items, dis_scores, color='lightcoral', alpha=0.8)
ax2.set_title('⚠️ 한계점 (낮을수록 문제)', fontsize=14, color='darkred')
ax2.set_xlabel('점수 (1-10)')
ax2.set_xlim(0, 10)
ax2.grid(True, alpha=0.3)

# 점수 레이블
for bar, score in zip(bars2, dis_scores):
    ax2.text(bar.get_width() + 0.1, bar.get_y() + bar.get_height()/2,
             f'{score}', va='center', fontweight='bold')

plt.tight_layout()
plt.show()

# 종합 평가
total_advantages = sum(advantages.values())
total_disadvantages = sum(disadvantages.values())
overall_score = total_advantages / (total_advantages + (80 - total_disadvantages))

print(f"\n📊 종합 평가")
print(f"장점 총점: {total_advantages}/80 ({total_advantages/80*100:.1f}%)")
print(f"단점 보완: {total_disadvantages}/80 ({total_disadvantages/80*100:.1f}%)")
print(f"종합 점수: {overall_score:.3f} ({overall_score*100:.1f}%)")

## 🎯 결론 및 권장사항

In [ ]:
# 최종 권장사항 요약
print("🎯 Decision Tree 결론 및 권장사항")
print("="*60)

print("\n🏆 핵심 성과:")
achievements = [
    f"✅ 우수한 성능: {test_results['accuracy']*100:.2f}% 정확도",
    f"⚡ 빠른 훈련: {training_results['total_training_time']/60:.1f}분",
    f"🔍 완전한 해석가능성: {len(tree_rules)}개 규칙 제공",
    f"📊 완벽한 AUC: {test_results['auc_score']*100:.2f}%",
    "🚀 실시간 추론 가능"
]

for achievement in achievements:
    print(f"  {achievement}")

print("\n💼 추천 사용 사례:")
recommendations = [
    "🏢 기업 환경: 투명한 의사결정이 중요한 경우",
    "🏛️ 규제 산업: 금융, 의료, 정부 기관",
    "📚 교육 목적: AI 의사결정 과정 설명",
    "⚡ 실시간 시스템: 빠른 응답이 필요한 환경",
    "🔧 프로토타이핑: 빠른 개발과 검증"
]

for recommendation in recommendations:
    print(f"  {recommendation}")

print("\n⚠️ 주의사항:")
cautions = [
    "복잡한 비선형 패턴에는 한계 존재",
    "과적합 방지를 위한 정기적인 검증 필요",
    "새로운 공격 패턴에 대한 규칙 업데이트 필요",
    "극도로 복잡한 경우 다른 앙상블 방법 고려"
]

for caution in cautions:
    print(f"  ⚠️ {caution}")

print("\n🌟 최종 결론:")
print("Decision Tree는 해석가능성과 성능의 균형이 뛰어난")
print("실용적인 선택으로, 특히 투명성이 중요한 환경에서")
print("탁월한 성능을 보여줍니다.")

print(f"\n📈 성능 지표 요약:")
print(f"  🎯 정확도: {test_results['accuracy']*100:.4f}%")
print(f"  ⚡ 효율성: {training_results['total_training_time']/60:.1f}분 훈련")
print(f"  🔍 해석성: {len(tree_rules)}개 명확한 규칙")
print(f"  📊 신뢰도: {test_results['auc_score']*100:.2f}% AUC")